In [2]:
import warnings
warnings.filterwarnings("ignore")
import numpy as np
import pandas as pd
#导入数据
data_control = pd.read_excel('E:\Huawei cloud\华为云盘\文档\毕业论文\stata代码\data_control.xlsx')
data_treat = pd.read_excel('E:\Huawei cloud\华为云盘\文档\毕业论文\stata代码\data_treat.xlsx')

In [3]:
data_treat['action'] = data_treat['Year'].where(data_treat['Vc']==True)
data_treat['action'] = data_treat.groupby('Stkcd')['action'].transform('min')
treat_pre = data_treat[data_treat['Year']<data_treat['action']]
treat_post = data_treat[data_treat['Year']>=data_treat['action']]
treat_pre.head()

,Stkcd,Year,Vc,IND,Caperc,Lev,PPEperc,NPGR,SGR,RTR,ROA,Lnpatent,Lnsize,action
0,12,2004,False,建筑材料,0.115335,0.481675,0.982086,0.698250,0.088428,9.292599,0.089352,0.693147,22.279432,2015.0
1,12,2005,False,建筑材料,0.105231,0.508313,0.985549,0.026604,0.077745,8.583376,0.073790,1.609438,22.432246,2015.0
2,12,2006,False,建筑材料,0.082842,0.565768,0.978328,0.092974,-0.014432,9.299277,0.067324,1.945910,22.636097,2015.0
3,12,2007,False,建筑材料,0.455311,0.493006,0.964869,0.370526,0.099847,13.703556,0.074495,2.890372,22.857785,2015.0
4,12,2008,False,建筑材料,0.123064,0.536142,0.970028,-0.118153,0.084869,15.531408,0.053134,2.197225,23.062767,2015.0


In [4]:
treat_pre = treat_pre.drop(['action'],axis=1)
df = pd.concat([data_control,treat_pre],axis=0)
df = df.drop(['Vc'],axis=1)
df.head()

,Stkcd,Year,IND,Caperc,Lev,PPEperc,NPGR,SGR,RTR,ROA,Lnpatent,Lnsize
0,2,2004,房地产,0.324336,0.594163,1.0,0.612774,0.099708,12.736188,0.069947,2.833213,23.466324
1,2,2005,房地产,0.361138,0.609809,1.0,0.570613,0.113548,10.997000,0.076395,1.609438,23.813962
2,2,2006,房地产,0.981768,0.649418,1.0,0.503140,0.096677,16.854687,0.061124,3.637586,24.604993
3,2,2007,房地产,0.943422,0.661125,1.0,1.467931,0.158089,57.790683,0.070893,0.693147,25.329380
4,2,2008,房地产,0.144431,0.674441,1.0,-0.127434,0.117774,45.860878,0.042309,3.295837,25.504375


In [5]:
from sklearn.preprocessing import StandardScaler
df_r = df[['Caperc','Lev','PPEperc','NPGR','SGR','RTR','ROA','Lnsize','Lnpatent']]
scaler = StandardScaler().fit(df_r)
df[['Caperc','Lev','PPEperc','NPGR','SGR','RTR','ROA','Lnsize','Lnpatent']] = scaler.transform(df_r)
df.head()

,Stkcd,Year,IND,Caperc,Lev,PPEperc,NPGR,SGR,RTR,ROA,Lnpatent,Lnsize
0,2,2004,房地产,-0.007413,0.802780,0.779348,-0.007271,0.092017,-0.029402,0.299939,0.151107,1.178013
1,2,2005,房地产,0.018697,0.878237,0.779348,-0.007382,0.121104,-0.030076,0.409323,-0.653491,1.439538
2,2,2006,房地产,0.459028,1.069261,0.779348,-0.007559,0.085647,-0.027805,0.150264,0.679959,2.034625
3,2,2007,房地产,0.431822,1.125721,0.779348,-0.005026,0.214715,-0.011935,0.315987,-1.255927,2.579576
4,2,2008,房地产,-0.135054,1.189941,0.779348,-0.009214,0.129986,-0.016560,-0.168915,0.455269,2.711224


In [6]:
#对Lnsize和Lnpatent进行缩尾处理
# 定义一个缩尾处理函数
def winsorize(series, lower_percentile=0.05, upper_percentile=0.95):
    lower_limit = np.percentile(series, lower_percentile * 100)
    upper_limit = np.percentile(series, upper_percentile * 100)
    return series.clip(lower=lower_limit, upper=upper_limit)

columns_to_winsorize = ['Lnpatent', 'Lnsize']

# 对每一列进行缩尾处理
for col in columns_to_winsorize:
    df[col] = winsorize(df[col], lower_percentile=0.05, upper_percentile=0.95)

In [7]:
from sklearn.preprocessing import LabelEncoder
label_encoder_stkcd = LabelEncoder()
df['Stkcd'] = label_encoder_stkcd.fit_transform(df['Stkcd'])
label_encoder_ind = LabelEncoder()
df['IND']= label_encoder_ind.fit_transform(df['IND'])
df.head()

,Stkcd,Year,IND,Caperc,Lev,PPEperc,NPGR,SGR,RTR,ROA,Lnpatent,Lnsize
0,0,2004,11,-0.007413,0.802780,0.779348,-0.007271,0.092017,-0.029402,0.299939,0.151107,1.178013
1,0,2005,11,0.018697,0.878237,0.779348,-0.007382,0.121104,-0.030076,0.409323,-0.653491,1.439538
2,0,2006,11,0.459028,1.069261,0.779348,-0.007559,0.085647,-0.027805,0.150264,0.679959,1.866277
3,0,2007,11,0.431822,1.125721,0.779348,-0.005026,0.214715,-0.011935,0.315987,-1.255927,1.866277
4,0,2008,11,-0.135054,1.189941,0.779348,-0.009214,0.129986,-0.016560,-0.168915,0.455269,1.866277


In [8]:
num_stocks=df['Stkcd'].nunique()
print(num_stocks)

2321


In [9]:
from pytorch_forecasting import TimeSeriesDataSet